In [1]:
positions = [
    'pours',
    'contres',
    'abstentions'
]

In [2]:
import json
import pandas as pd
from glob import glob

In [3]:
def get_scrutin(filename):
    return json.load(open(filename))

scrutins = [ get_scrutin(f) for f in glob('data/**/VTAN*.json') ]

len(scrutins)

1504

In [4]:
def get_votes(scrutin):
    data = scrutin
    groupes = data['scrutin']['ventilationVotes']['organe']['groupes']['groupe']

    #print(scrutin)
    
    votes = [ (p, g['organeRef'], g['vote']['decompteNominatif'][p]['votant'], g['vote']['positionMajoritaire'])
           for g in groupes
           for p in positions
           if g['vote']['decompteNominatif'][p] != None
          ]

    df = (
        pd.concat([
            pd
            .DataFrame
            .from_records([v] if not isinstance(v, list) else v)
            .assign(
                position=p,
                organe=o,
                date=scrutin['scrutin']['dateScrutin'],
                scrutin=scrutin['scrutin']['uid'],
                groupe_majorite= gv
            )
            for (p, o, v, gv) in votes
        ])
        .assign(
            position = lambda df: df.position.replace({'pours': 'pour', 'contres': 'contre', 'abstentions': 'abstention'})
        )
    )

    return df

votes = pd.concat([ get_votes(scrutin) for scrutin in scrutins ])

votes

,acteurRef,mandatRef,parDelegation,position,organe,date,scrutin,groupe_majorite
0,PA609520,PM797676,false,contre,PO800538,2022-12-08,VTANR5L16V703,contre
1,PA722252,PM797610,false,contre,PO800538,2022-12-08,VTANR5L16V703,contre
2,PA605963,PM796494,false,contre,PO800538,2022-12-08,VTANR5L16V703,contre
3,PA721852,PM797565,true,contre,PO800538,2022-12-08,VTANR5L16V703,contre
4,PA718930,PM796287,false,contre,PO800538,2022-12-08,VTANR5L16V703,contre
...,...,...,...,...,...,...,...,...
2,PA795184,PM797346,false,abstention,PO800502,2022-10-28,VTANR5L16V452,abstention
3,PA795998,PM797796,false,abstention,PO800502,2022-10-28,VTANR5L16V452,abstention
4,PA796010,PM797802,false,abstention,PO800502,2022-10-28,VTANR5L16V452,abstention
5,PA774962,PM797766,false,abstention,PO800502,2022-10-28,VTANR5L16V452,abstention


In [5]:
votes.to_csv('out/votes.csv', index=False)

In [6]:
votes.to_parquet('out/votes.par', index=False)

In [7]:
scrutins_df = pd.DataFrame.from_records([
    {
        'uid': s['scrutin']['uid'],
        'objet': s['scrutin']['titre'],
        'type': s['scrutin']['typeVote']['libelleTypeVote'],
        'date': s['scrutin']['dateScrutin']
    }
    for s in scrutins
])

scrutins_df

,uid,objet,type,date
0,VTANR5L16V703,l'amendement n° 1419 (rect.) de M. Jumel à l'a...,scrutin public ordinaire,2022-12-08
1,VTANR5L16V1359,l'amendement n° 93 de Mme Engrand à l'article ...,scrutin public ordinaire,2023-03-30
2,VTANR5L16V353,la première partie du projet de loi de finance...,scrutin public ordinaire,2022-10-20
3,VTANR5L16V646,l'amendement n° 124 de M. Chenu après l'articl...,scrutin public ordinaire,2022-11-29
4,VTANR5L16V216,l'amendement n° 41 de M. Delaporte après l'art...,scrutin public ordinaire,2022-10-05
...,...,...,...,...
1499,VTANR5L16V844,l'article premier de la proposition de loi vis...,scrutin public ordinaire,2023-01-18
1500,VTANR5L16V901,l'amendement n° 8 de M. Bovet à l'article prem...,scrutin public ordinaire,2023-01-31
1501,VTANR5L16V1008,l'amendement n° 3606 de Mme Amiot et les amend...,scrutin public ordinaire,2023-02-16
1502,VTANR5L16V1458,l'amendement n° 9 de Mme Paris à l'article pre...,scrutin public ordinaire,2023-05-04


In [8]:
scrutins_df.type.value_counts()

type
scrutin public ordinaire    1474
scrutin public solennel       30
Name: count, dtype: int64

In [9]:
scrutins_df.to_csv('out/scrutins.csv', index=False)